In [11]:
# from Utills import TrainFX
import json
from requests.exceptions import Timeout
import requests
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import date, datetime,timedelta
train_info_list = []

max_attempts = 3  # Maximum number of retry attempts

faulty_train = []


In [14]:
class TrainFX():
    def getTrainList(self):
        response = requests.get('http://www.indianrail.gov.in/enquiry/FetchTrainData?_=1557752279434')
        datastore = json.loads(response.text)
        train_numbers = []
        for tr in datastore:
            train_numbers.append(tr.split('-')[0].strip())
        
        print("Found {} trains".format(len(train_numbers)))
        return train_numbers
    
    def getTrainInfo(self, train_number, timeout):
        unix_time = int(datetime.utcnow().timestamp()-1532)
        cookies = {
            '__gads' : 'ID=0b5cc381a6cc5d5e:T=1697635642:RT=1697635642:S=ALNI_MbSIbTJTVGP3rMCM8Xt4aDIub5Evg',
            '_gid' : 'GA1.3.1679543960.1697635644; _ga_SHTZYKNHG2=GS1.1.1697635639.1.1.1697635683.0.0.0',
            'JSESSIONID' : 'GA9C-MwGVJERPBL6MlhhbR39HY-Hq--Fog7TVrg5Lltr7FNiAraG!45373394; _ga=GA1.1.1336431025.1697635639',
        }
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
            'Accept': 'application/json, text/plain, */*',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.irctc.co.in/nget/booking/check-train-schedule',
            'Content-Type': 'application/x-www-form-urlencoded',
            'greq': str(unix_time),
            'Content-Language': 'en',
            'Connection': 'keep-alive',
        }
        
        response = requests.get('https://www.irctc.co.in/eticketing/protected/mapps1/trnscheduleenquiry/'+train_number, headers=headers, timeout= timeout)
        return response
    
    def getTrainDelay(self, train_number = '', timeout=10):
        unix_time = int(datetime.utcnow().timestamp()-1532)
        cookies = {
            '__gads' : 'ID=0b5cc381a6cc5d5e:T=1697635642:RT=1697635642:S=ALNI_MbSIbTJTVGP3rMCM8Xt4aDIub5Evg',
            '_gid' : 'GA1.3.1679543960.1697635644; _ga_SHTZYKNHG2=GS1.1.1697635639.1.1.1697635683.0.0.0',
            'JSESSIONID' : 'GA9C-MwGVJERPBL6MlhhbR39HY-Hq--Fog7TVrg5Lltr7FNiAraG!45373394; _ga=GA1.1.1336431025.1697635639',
        }
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
            # 'Referer': 'https://www.irctc.co.in/nget/booking/check-train-schedule',
            'Content-Type': 'text/html; charset=UTF-8',
            'greq': str(unix_time),
            # 'Content-Language': 'en',
            'Connection': 'keep-alive',
        }
        url = f"https://etrain.info/train/{train_number}/history?d=3"
        response = requests.get(url, headers=headers, timeout= timeout)
        soup = BeautifulSoup(response.content, 'html.parser')
        pattern = re.compile(r'\((.*?)\)')
        section_links = soup.find_all('a', class_='runStatStn')
        section_data = []
        station_delay = {}
        for link in section_links:
            section_name = link.find('div').text.strip()
            avg_delay_text = link.find('div', class_='inlineblock pdl5').text
            avg_delay = int(avg_delay_text.split(':')[1].strip().split(' ')[0])
            section_data.append({'section_name': section_name, 'avg_delay': avg_delay})
            clean_section_name = section_name.replace("\r\n\t\t\t\t\t\t\t\t\nAvg. Delay: ", '')
            station_delay[clean_section_name] = avg_delay

        # for data in section_data:
        #     print(f"Section: {data['section_name']}, Avg. Delay: {data['avg_delay']} Min's")
        station_delay = {pattern.search(key).group(1): value for key, value in station_delay.items()}
        return response, station_delay
    
t = TrainFX()
trainname = t.getTrainList()

Found 5591 trains


In [ ]:

for train in trainname:
    attempts = 0
    while attempts < max_attempts:
        try:
            response = t.getTrainInfo(train, timeout=3)  # Set an appropriate timeout value
            response.raise_for_status()  # Raise an HTTPError for bad responses
            train_info = json.loads(response.text)
            print("Train: ", train, " Info received")
            if 'errorMessage' not in train_info:
                train_info_list.append(train_info)
            else:
                faulty_train.append(train)
                print(f"Faulty train number {train}")
            break  # Break out of the retry loop if successful
        except Timeout as e:
            print(f"Timeout error for train {train}. Retrying... ({attempts + 1}/{max_attempts})")
            attempts += 1
        except requests.RequestException as e:
            print(f"Request failed for train {train} with error: {e}")
            break  # Break out of the loop for other request errors

    if attempts == max_attempts:
        print(f"Error: Max attempts reached for train {train}")

# store the train_info_list in a new file
with open('train_info.json', 'w') as f:
    json.dump(train_info_list, f)




#### Train Delay Scrapping

In [29]:
train_delay_response = t.getTrainDelay('12807')
print(train_delay_response)

{'VSKP': 6, 'SCM': 4, 'VZM': 6, 'VBL': 17, 'PVP': 6, 'PVPT': 5, 'RGDA': 8, 'MNGD': 20, 'AMB': 24, 'KSNG': 31, 'TIG': 32, 'KBJ': 27, 'HSK': 26, 'KRAR': 24, 'BGBR': 24, 'MSMD': 24, 'R': 5, 'DURG': 1, 'RJN': 18, 'DGG': 24, 'G': 30, 'TMR': 34, 'BRD': 38, 'NGP': 31, 'MTY': 46, 'BZU': 51, 'ET': 26, 'BPL': 33, 'BINA': 25, 'LAR': 43, 'VGLJ': 29, 'GWL': 67, 'AGC': 47, 'RKM': 51, 'MTJ': 66, 'FDB': 59, 'NZM': 54}


In [ ]:
train_info = ''
train_with_no_delay_data = []
train_delay_dict = {}
# trainname = trainname[:30]
for train in trainname:
    attempts = 0
    while attempts < max_attempts:
        try:
            response, train_delay = t.getTrainDelay(train, timeout=3)  # Set an appropriate timeout value
            response.raise_for_status()  # Raise an HTTPError for bad responses
            print("Train: ", train, " Info received")
            if 'errorMessage' not in train_info:
                train_delay_dict[train] = train_delay
            else:
                faulty_train.append(train)
                print(f"Faulty train number {train}")
            break  # Break out of the retry loop if successful
        except Timeout as e:
            print(f"Timeout error for train {train}. Retrying... ({attempts + 1}/{max_attempts})")
            attempts += 1
        except requests.RequestException as e:
            train_with_no_delay_data.append(train)
            print(f"Request failed for train {train} with error: {e}")
            break  # Break out of the loop for other request errors

    if attempts == max_attempts:
        print(f"Error: Max attempts reached for train {train}")

# store the train_info_list in a new file
with open('train_average_delay.json', 'w') as f:
    json.dump(train_delay_dict, f)


In [11]:
print(train_with_no_delay_data)

['01047', '07034', '14817', '08501', '07227', '07532', '01080', '59077', '01142', '09471', '16557', '01288', '07405', '03288', '07138', '05557', '07647', '03173', '07478', '01279', '07090', '08305', '05781', '07401', '08624', '03014', '11104', '05552', '54834', '02515', '01059', '01078', '07431', '01585', '03287', '01234', '05887', '02552', '07038', '05531', '06057', '18804', '06437', '09248', '01245', '07180', '07620', '16852', '07648', '08626', '01406', '06175', '04836', '01064', '01262', '09179', '11086', '09445', '02195', '07134', '08478', '22477', '58013', '19336', '02411', '01462', '01110', '59207', '08026', '52587', '19064', '19419', '05119', '02730', '03134', '17203', '02001', '13575', '07183', '08402', '02194', '01007', '06251', '01107', '08616', '11111', '07353', '06553', '09446', '56502', '55663', '02891', '01257', '53050', '02181', '02097', '07571', '09188', '01294', '07048', '08568', '07373', '18191', '03391', '56501', '07253', '55664', '07141', '07473', '01231', '07386', 

In [14]:
def getTrainDelayOnDate(train_no, date, timeout = 4):
    unix_time = int(datetime.utcnow().timestamp()-1532)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
        # 'Referer': 'https://www.irctc.co.in/nget/booking/check-train-schedule',
        'Content-Type': 'text/html; charset=UTF-8',
        'greq': str(unix_time),
        # 'Content-Language': 'en',
        'Connection': 'keep-alive',
    }
    base_url = "https://runningstatus.in/check.php"
    params = {
        'a': 'history',
        'trainno': train_no,
        'duration': 'seldates',
        'fdate': date,
        'tdate': date
    }
    response = requests.get(base_url, params=params, timeout = timeout)
    soup = BeautifulSoup(response.content, 'html.parser')
    delay_table = soup.find('table', class_='table-striped')
    if delay_table is None :
        return response, {}
    date_station_delay = {}
    tds = delay_table.find_all('td', {'class': None, 'style': None})
    stations = tds[::2]  # Select every second td element
    delays = delay_table.find_all('span', class_='text-danger')
    delays = [td.find('span', class_='text-danger') for td in tds[1::2]]

    for station, delay in zip(stations, delays):
        station_name = station.find('abbr').get('title')
        distance = station.find('small').text
        delay_time = delay.text.strip() if delay else ''
        date_station_delay[station_name] = delay_time
    
    return response, date_station_delay

In [16]:
def getDelayForAllDates(train_no):
    # https://runningstatus.in/check.php?a=history&trainno=12145&duration=seldates&fdate=03-11-2023&tdate=04-11-2023
    
    start_date = date(2022, 11, 1)
    end_date = date(2023, 10, 31)

    current_date = start_date
    train_delay_dict = {}
    while current_date <= end_date:
        formatted_date = current_date.strftime("%d-%m-%Y")
        attempts = 0
        while attempts < max_attempts:
            try:
                response, train_date_dict = getTrainDelayOnDate(train_no, formatted_date)  # Set an appropriate timeout value
                response.raise_for_status()  # Raise an HTTPError for bad responses
                print("Train: ", train_no, " delay for ", formatted_date, " received")
                train_delay_dict[formatted_date] = train_date_dict
                break  # Break out of the retry loop if successful
            except Timeout as e:
                print(f"Timeout error for train {train}. Retrying... ({attempts + 1}/{max_attempts})")
                attempts += 1
            except requests.RequestException as e:
                train_with_no_delay_data.append(train)
                print(f"Request failed for train {train} with error: {e}")
                break  # Break out of the loop for other request errors
            if attempts == max_attempts:
                print(f"Error: Max attempts reached for train {train}")
        
        current_date += timedelta(days=1)
    return train_delay_dict

In [10]:
# Load the JSON file
with open('train_info.json', 'r') as file:
    data = json.load(file)

# Extract values for the key "train number"
train_numbers = [entry.get("trainNumber") for entry in data]

# Print or use the list of train numbers as needed
print(train_numbers)
print(len(train_numbers))

['12303', '17020', '14233', '16232', '22101', '22968', '18628', '22164', '16305', '12822', '14208', '13225', '12740', '11274', '13331', '20658', '12341', '19611', '04121', '16381', '17248', '13404', '12314', '12966', '12709', '22671', '12153', '20632', '12849', '12032', '12735', '17249', '19435', '16606', '05061', '11108', '22852', '22623', '12019', '14614', '12672', '22422', '12787', '18449', '15656', '22867', '17202', '01066', '22967', '13064', '20979', '18636', '17407', '12292', '17226', '12524', '03247', '12834', '16175', '12371', '14054', '12879', '13043', '12369', '18301', '22501', '14717', '22106', '12240', '07255', '22605', '14814', '12266', '07091', '20824', '22724', '12881', '13154', '06142', '22302', '22995', '20704', '12080', '16615', '22199', '20665', '22666', '19271', '17261', '22468', '22679', '19484', '12439', '12594', '08580', '18504', '14316', '15617', '18101', '14865', '15633', '18635', '07411', '12228', '12695', '12965', '22151', '18530', '12567', '12176', '22428', 

In [12]:
# di, st = getTrainDelayOnDate('15018', '04-10-2023')
# print(st) 
t = getDelayForAllDates('22436')
with open('15018_delay_oneYear.json', 'w') as f:
    json.dump(t, f)

Train:  22436  delay for  01-11-2022  received
Train:  22436  delay for  02-11-2022  received
Train:  22436  delay for  03-11-2022  received
Train:  22436  delay for  04-11-2022  received
Train:  22436  delay for  05-11-2022  received
Train:  22436  delay for  06-11-2022  received
Train:  22436  delay for  07-11-2022  received
Train:  22436  delay for  08-11-2022  received


KeyboardInterrupt: 

In [17]:
import numpy as np
final_train_delay_data = {}
num_parts = 20
split_arrays = np.array_split(train_numbers, num_parts)
train_with_no_delay_data = []
for i in range(num_parts):
    train_numbers = split_arrays[i]
    for train in train_numbers:
        t = getDelayForAllDates(train)
        if t == {}:
            train_with_no_delay_data.append(train)
            continue
        final_train_delay_data[train] = t
        with open(f'train_delay_22_23_{i + 1}', 'w') as f:
            json.dump(final_train_delay_data, f)

Train:  12303  delay for  01-11-2022  received
Train:  12303  delay for  02-11-2022  received
Train:  12303  delay for  03-11-2022  received
Train:  12303  delay for  04-11-2022  received
Train:  12303  delay for  05-11-2022  received
Train:  12303  delay for  06-11-2022  received
Train:  12303  delay for  07-11-2022  received
Train:  12303  delay for  08-11-2022  received
Train:  12303  delay for  09-11-2022  received
Train:  12303  delay for  10-11-2022  received
Timeout error for train 12303. Retrying... (1/3)
Train:  12303  delay for  11-11-2022  received
Train:  12303  delay for  12-11-2022  received
Train:  12303  delay for  13-11-2022  received
Train:  12303  delay for  14-11-2022  received
Train:  12303  delay for  15-11-2022  received
Train:  12303  delay for  16-11-2022  received
Train:  12303  delay for  17-11-2022  received
Train:  12303  delay for  18-11-2022  received
Train:  12303  delay for  19-11-2022  received
Train:  12303  delay for  20-11-2022  received
Train:  123

In [12]:
file = './train_list/prokerala_list.html'
with open(file, 'r') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

train_numbers = [td.text for td in soup.find_all('td', {'class': None})[1::5]]

print(train_numbers)
print(len(train_numbers))

['14722', '09438', '05348', '22188', '17409', '07775', '07851', '08653', '08659', '08643', '08657', '08655', '08661', '08647', '08641', '08671', '08669', '08674', '08676', '08684', '08663', '08686', '08680', '08649', '20501', '15626', '05675', '05628', '07679', '12097', '02502', '12520', '01666', '07682', '07684', '07690', '07688', '13174', '07029', '15663', '05695', '12504', '06627', '06629', '12341', '12342', '04167', '04165', '04164', '14011', '20976', '13168', '01909', '04495', '12547', '11808', '01972', '11902', '04157', '12195', '22988', '01913', '05414', '05914', '12180', '15055', '14211', '22645', '22646', '22185', '22186', '03056', '09274', '04168', '04166', '09400', '19435', '19483', '19418', '15560', '09465', '19165', '20947', '20949', '19489', '19409', '12917', '12833', '19223', '22925', '01906', '19413', '19401', '22920', '20954', '12932', '82902', '12010', '09435', '09447', '19421', '09417', '22967', '12297', '12844', '20862', '11049', '19415', '20939', '09419', '19036', 

In [18]:
index = train_numbers.index('07470')
train_numbers = train_numbers[index:]
print(len(train_numbers))

118


In [20]:
import json

train_list1 = []
with open('train_info.json', 'r') as file:
    data = json.load(file)
train_list1 = [entry for entry in data]

dict_set = set()

t = TrainFX()
for d in train_list1:
    dict_set.add(d['trainNumber'])

max_attempts = 3

for train in train_numbers:
    if train not in dict_set:
        attempts = 0
        while attempts < max_attempts:
            try:
                response = t.getTrainInfo(train, timeout=2)  # Set an appropriate timeout value
                response.raise_for_status()  # Raise an HTTPError for bad responses
                train_info = json.loads(response.text)
                if 'errorMessage' not in train_info:
                    print("Train: ", train, " Info received")
                    train_list1.append(train_info)
                else:
                    print(f"Faulty train number {train}")
                break  # Break out of the retry loop if successful
            except Timeout as e:
                print(f"Timeout error for train {train}. Retrying... ({attempts + 1}/{max_attempts})")
                attempts += 1
            except requests.RequestException as e:
                print(f"Request failed for train {train} with error: {e}")
                break  # Break out of the loop for other request errors

        if attempts == max_attempts:
            print(f"Error: Max attempts reached for train {train}")


Faulty train number 07470
Faulty train number 08532
Faulty train number 08528
Faulty train number 07467
Train:  18503  Info received
Train:  08583  Info received
Faulty train number 07468
Faulty train number 09150
Timeout error for train 07469. Retrying... (1/3)
Timeout error for train 07469. Retrying... (2/3)
Faulty train number 07469
Faulty train number 06891
Faulty train number 06121
Faulty train number 06895
Faulty train number 07751
Faulty train number 01382
Faulty train number 08808
Train:  09502  Info received
Train:  12251  Info received
Train:  12252  Info received
Faulty train number 09585
Faulty train number 09561
Faulty train number 09563
Faulty train number 09439
Faulty train number 09441
Faulty train number 09443
Faulty train number 07463
Faulty train number 01372
Faulty train number 01315
Faulty train number 01373
Faulty train number 11122
Train:  22637  Info received
Faulty train number 01766
Train:  12613  Info received
Train:  12614  Info received
Faulty train number 

In [21]:
print(len(train_list1))
with open('train_info.json', 'w') as f:
    json.dump(train_list1, f)

3397
